In [1]:
#! sudo apt install espeak
#!sudo apt install libcairo2-dev


In [2]:
# import sys
# sys.path.append("nix-tts")

In [3]:
# from nix.models.TTS import NixTTSInference
# from IPython.display import Audio

# # Initiate Nix-TTS
# nix = NixTTSInference(model_dir = "/home/miko/Downloads/nix-ljspeech-stochastic-v0.1")
# # Tokenize input text
# tts_text = """Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. This should be suitable for many users. Preview is available if you want the latest, not fully tested and supported, builds that are generated nightly. Please ensure that you have met the prerequisites below (e.g., numpy), depending on your package manager. Anaconda is our """

# c, c_length, phoneme = nix.tokenize(tts_text)
# # Convert text to raw speech
# xw = nix.vocalize(c, c_length)

# # Listen to the generated speech
# Audio(xw[0,0], rate = 22050)

In [4]:
# with open("assets/demo_png_2_vid/would_you_support_a_mandatory_retirement_age_of/Askmyrkr-1_7k/0001.txt","r") as f:
#     reddit_txt = f.read()

In [5]:
# nix = NixTTSInference(model_dir = "/home/miko/Downloads/nix-ljspeech-stochastic-v0.1")
# # Tokenize input text
# c, c_length, phoneme = nix.tokenize(reddit_txt)
# # Convert text to raw speech
# xw = nix.vocalize(c, c_length)

# # Listen to the generated speech
# Audio(xw[0,0], rate = 22050)

In [6]:
# reddit_txt

In [7]:
# import yaml

In [8]:
# yaml.dump

In [9]:
# from tortoise.api import TextToSpeech, MODELS_DIR
# from tortoise.utils.audio import load_voices
# from utils.text import split_and_recombine_text
# import torch
# import torchaudio
# import glob,os

In [10]:


# tts = TextToSpeech(models_dir=MODELS_DIR)

# #linux below
# #available_voices = [x.split("/")[-1] for x in glob.glob("tortoise-tts/tortoise/voices/*")]
# #windows below
# available_voices = [x.split("\\")[-1] for x in glob.glob("tortoise-tts/tortoise/voices/*")]
# #selected_voices = [available_voices[0]]
# selected_voices = available_voices
# tts_text ="""Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. This should be suitable for many users. Preview is available if you want the latest, not fully tested and supported, builds that are generated nightly. Please ensure that you have met the prerequisites below (e.g., numpy), depending on your package manager. Anaconda is our """
# output_path = "assets/demo_trts"
# os.makedirs(output_path, exist_ok=True)
# candidates=1
# cvvp_amount=0
# seed=None
# preset='fast' #standard
# selected_voices=["train_empire&applejack"]
# for k, selected_voice in enumerate(selected_voices):
#     if '&' in selected_voice:
#         voice_sel = selected_voice.split('&')
#     else:
#         voice_sel = [selected_voice]
#     voice_samples, conditioning_latents = load_voices(voice_sel)

#     gen, dbg_state = tts.tts_with_preset(tts_text, k=candidates, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
#                               preset=preset, use_deterministic_seed=seed, return_deterministic_state=True, cvvp_amount=cvvp_amount)
#     if isinstance(gen, list):
#         for j, g in enumerate(gen):
#             torchaudio.save(os.path.join(output_path, f'{selected_voice}_{k}_{j}_{len(tts_text)}.wav'), g.squeeze(0).cpu(), 24000)
#     else:
#         torchaudio.save(os.path.join(output_path, f'{selected_voice}_{k}_{len(tts_text)}.wav'), gen.squeeze(0).cpu(), 24000)

In [11]:
# from IPython.display import Audio

In [12]:
# Audio(filename="assets/demo_trts/angie_0_405.wav")

In [3]:
# demo processing reddit texts
from tortoise.api import TextToSpeech, MODELS_DIR
from tortoise.utils.audio import load_voices
from tortoise.utils.text import split_and_recombine_text
import torch
import torchaudio
import glob,os
import time
import re
from tqdm import tqdm

C:\Users\mko23\.virtualenvs\reddit-yt-tts-iR6L085J\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from tts_job_opt import TTSOPT

In [14]:
def exec_tts(tts_text,output_path,selected_voice="train_empire"):
    
    candidates=1
    cvvp_amount=0
    seed=None
    preset= 'fast' #'standard' #'fast' #standard
    
    texts = split_and_recombine_text(tts_text)
    
    if '&' in selected_voice:
        voice_sel = selected_voice.split('&')
    else:
        voice_sel = [selected_voice]
    voice_samples, conditioning_latents = load_voices(voice_sel)

    all_parts = []
    for j, text in enumerate(texts):
        gen, dbg_state = tts.tts_with_preset(text, k=candidates, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                              preset=preset, use_deterministic_seed=seed, return_deterministic_state=True, cvvp_amount=cvvp_amount,verbose=False)
        gen = gen.squeeze(0).cpu()
        all_parts.append(gen)
        
    full_audio = torch.cat(all_parts, dim=-1)
    torchaudio.save(os.path.join(output_path), full_audio, 24000)

In [16]:
completed_dir = "N:/data/reddit-yt-tts/completed"
archive_text_dir = "N:/data/reddit-yt-tts/archive_text"
queue_dir = "N:/data/reddit-yt-tts/queued"
post_dir = os.listdir(queue_dir)

tts = TextToSpeech(models_dir=MODELS_DIR)

start_time = time.time()

skip_posts = ["what_famous_person_didnt_deserve_all_the_hate"]
for post in post_dir:
    print(post)
    if post in skip_posts:
        continue
    thread_dir = os.listdir(f"{queue_dir}/{post}")
    i = 0
    cmt_threshold = 35
    for thread in tqdm(thread_dir):
        if ".mp4" in thread:
            continue
        
        text_files = [x for x in os.listdir(f"{queue_dir}/{post}/{thread}") if x[-3:] == "txt" ]
        for text_file in text_files:
           
            # read in text file
            text_path = f"{queue_dir}/{post}/{thread}/{text_file}"
            with open(text_path,"r",encoding="utf8") as f:
                tts_text = f.read()
                #remove emojis https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
                emoji_pattern = re.compile("["
                                                         u"\U0001F600-\U0001F64F"  # emoticons
                                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                        u"\U00002500-\U00002BEF"  # chinese char
                                        u"\U00002702-\U000027B0"
                                        u"\U00002702-\U000027B0"
                                        u"\U000024C2-\U0001F251"
                                        u"\U0001f926-\U0001f937"
                                        u"\U00010000-\U0010ffff"
                                        u"\u2640-\u2642" 
                                        u"\u2600-\u2B55"
                                        u"\u200d"
                                        u"\u23cf"
                                        u"\u23e9"
                                        u"\u231a"
                                        u"\ufe0f"  # dingbats
                                        u"\u3030"
                                               "]+", flags=re.UNICODE)
                tts_text = emoji_pattern.sub(r'', tts_text)
                tts_text = tts_text.strip()
                tts_text = tts_text.replace("“","").replace("”","").replace("―","")
                tts_text = tts_text.replace("☠️","")
                tts_text = tts_text.replace("[","")
                tts_text = tts_text.replace("]","")
                tts_text = tts_text.replace("/","")
                tts_text = tts_text.replace("%"," percent ")

            # save audio tts
            wav_file = text_file.replace("txt","wav")    
            output_dir = f"{completed_dir}/{post}/{thread}"
            os.makedirs(output_dir, exist_ok=True)
            output_path = f"{output_dir}/{wav_file}"
            exec_tts(tts_text,output_path)

            # archive text file incase reprocessing needed
            archive_dir = f"{archive_text_dir}/{post}/{thread}"
            os.makedirs(archive_dir, exist_ok=True)
            archive_path = f"{archive_dir}/{text_file}"
            with open(archive_path,"w",encoding="utf8") as f:
                f.write(tts_text)

            # remove processed text file (prevent uneeded reprocessing)
            os.remove(text_path)
            
        i+=1
        if i >= cmt_threshold:
            break
end_time = time.time()
#print(f"Done Processing took: {(end_time-start_time)/60} mins")

has_anyone_ever_been_to_a_wedding_where_someone


 61%|█████████████████████████████████████████████████▊                                | 34/56 [00:00<00:00, 75.26it/s]


people_who_get_in_their_cars_but_then_take_ages


100%|█████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 293.87it/s]


people_who_wake_up_at_5am_daily_why


...

 81%|█████████████████████████████████████████████████████████████████▌               | 34/42 [00:00<00:00, 299.04it/s]


what_television_series_had_the_biggest_bullshit


 68%|███████████████████████████████████████████████████████                          | 34/50 [00:00<00:00, 391.85it/s]


what_was_normal_2030_years_ago_but_is_considered


 77%|███████████████████████████████████████████████████████████████▎                  | 34/44 [45:13<13:17, 79.80s/it]


In [17]:
print(f"Done Processing took: {(end_time-start_time)/60} mins")

Done Processing took: 45.23986907402674 mins


In [ ]:
tts_text.replace("%"," percent ").replace("$","").replace(":","").replace("'","").replace("-","")

In [ ]:
import inspect
help(tts.tts_with_preset)

In [ ]:
import os
queue_dir = "N:/data/reddit-yt-tts/queued"
post = "whats_a_story_where_the_bad_guys_are_actually"
thread_dir = os.listdir(f"{queue_dir}/{post}")

In [ ]:
thread_dir